## imports

In [2]:
import tensorflow as tf 
import numpy as np
import keras.backend as K

import sys
sys.path.append('/opt/code/face/dual_shot')
import numpy as np
from net.dual_shot import test_net
from prepare_data.generator import image_reader,augment
from prepare_data.augment import Augmentation
from prepare_data.widerface import WIDERFaceDetection
from dual_conf import current_config as conf

config = tf.ConfigProto()
config.gpu_options.allow_growth= True
sess = tf.Session(config=config)
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

using: resnet50 as the base model


In [36]:
cls_target = np.random.randint(-1,2,[2,8])
cls_target

array([[ 1, -1,  1,  1,  1,  0, -1, -1],
       [-1,  1, -1,  1,  0, -1,  1,  0]])

In [37]:
predict_logits = np.random.randint(-1,10,[2,8,2])
predict_logits

array([[[ 8,  6],
        [ 1,  9],
        [ 2,  3],
        [ 4,  8],
        [ 3,  5],
        [ 7,  1],
        [ 6,  7],
        [ 4, -1]],

       [[ 5,  0],
        [-1,  7],
        [ 8,  3],
        [ 3,  7],
        [ 0,  8],
        [ 1,  7],
        [-1,  8],
        [ 3,  3]]])

In [38]:
cls_target = tf.constant(cls_target)
predict_logits = tf.constant(predict_logits,tf.float64)

In [27]:
train_indices = tf.where(tf.not_equal(cls_target, 0))
cls_target = tf.gather_nd(cls_target, train_indices)
cls_target = tf.where(cls_target > 0, cls_target, tf.zeros_like(cls_target))
cls_target = tf.one_hot(cls_target, depth=2)

In [28]:
# remove un-trained anchors from pred logit
logit0 = tf.gather_nd(predict_logits[..., 0], train_indices)
logit1 = tf.gather_nd(predict_logits[..., 1], train_indices)
predict_logits = tf.stack([logit0, logit1], axis=1)
predict_logits = tf.cast(predict_logits, dtype=tf.float64)

In [29]:
to_run = [train_indices,cls_target,predict_logits]
for i in to_run:
    print(i,'\n',sess.run(i))

Tensor("Where_33:0", shape=(?, 2), dtype=int64) 
 [[0 0]
 [0 1]
 [0 3]
 [0 5]
 [0 7]
 [1 0]
 [1 1]
 [1 3]
 [1 5]]
Tensor("one_hot_13:0", shape=(?, 2), dtype=float32) 
 [[0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
Tensor("stack_13:0", shape=(?, 2), dtype=float64) 
 [[-1.  4.]
 [ 1.  6.]
 [ 1.  7.]
 [ 6.  9.]
 [ 4.  5.]
 [ 1.  5.]
 [ 4.  8.]
 [ 5.  4.]
 [ 1.  9.]]


In [8]:
# loss_to_rank = log_sum_exp(cls_target,predict_logits)
# to_run = [loss_to_rank]
# for i in to_run:
#     print(i,'\n',sess.run(i))

In [9]:
cls_target,pred_logit = cls_target,predict_logits
logit_max = tf.reduce_max(pred_logit)
loss = tf.log(tf.reduce_sum(tf.exp(pred_logit - logit_max), 1, keepdims=True)) + logit_max
cls_indice = tf.where(cls_target>=1) 
anchor_cls_logit = tf.gather_nd(pred_logit,cls_indice)
loss = tf.squeeze(loss)
loss = loss - anchor_cls_logit


to_run = [pred_logit,cls_target,anchor_cls_logit,loss]
for i in to_run:
    print(i,'\n',sess.run(i))

Tensor("stack:0", shape=(?, 2), dtype=float64) 
 [[ 2.  5.]
 [-1.  1.]
 [ 8.  3.]
 [-1.  6.]
 [ 2.  0.]
 [ 4.  2.]
 [ 3.  4.]]
Tensor("one_hot:0", shape=(?, 2), dtype=float32) 
 [[1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Tensor("GatherNd_3:0", shape=(?,), dtype=float64) 
 [ 2.  1.  8. -1.  0.  2.  4.]
Tensor("sub_1:0", dtype=float64) 
 [3.04858735e+00 1.26928011e-01 6.71534849e-03 7.00091147e+00
 2.12692801e+00 2.12692801e+00 3.13261688e-01]


In [10]:
loss = tf.cast(cls_target[:,0],tf.float64)*loss # set loss for pos anchor to 0
_, neg_ind  = tf.nn.top_k(loss, 2)
pos_ind = tf.squeeze(tf.where(cls_target[:,1]>=1))
pos_ind = tf.cast(pos_ind,tf.int64)
neg_ind = tf.cast(neg_ind,tf.int64)
all_ind = tf.concat([pos_ind,neg_ind],axis=0)
# cls_target = tf.gather(cls_target,all_ind)
# pred_logit = tf.gather(pred_logit,all_ind)
to_run = [all_ind,cls_target,pred_logit,tf.gather(cls_target,all_ind),tf.gather(pred_logit,all_ind)]
for i in to_run:
    print(i,'\n',sess.run(i))

Tensor("concat:0", dtype=int64) 
 [1 4 5 6 3 0]
Tensor("one_hot:0", shape=(?, 2), dtype=float32) 
 [[1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Tensor("stack:0", shape=(?, 2), dtype=float64) 
 [[ 2.  5.]
 [-1.  1.]
 [ 8.  3.]
 [-1.  6.]
 [ 2.  0.]
 [ 4.  2.]
 [ 3.  4.]]
Tensor("GatherV2:0", dtype=float32) 
 [[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]
Tensor("GatherV2_1:0", dtype=float64) 
 [[-1.  1.]
 [ 2.  0.]
 [ 4.  2.]
 [ 3.  4.]
 [-1.  6.]
 [ 2.  5.]]


## func

In [3]:
# -*- coding:utf-8 _*-
"""
@author: danna.li
@date: 2019/4/25
@file: loss_func.py
@description:
"""

import tensorflow as tf
import keras.backend as K
from dual_conf import current_config as conf


def log_sum_exp(cls_target, predict_logits):
    """un-averaged confidence loss across all examples in a batch.
    :param cls_target: 2-d tensor,[num_anchors,2 ]; 1,0 for pos,neg anchors respectively
    :param predict_logits: 2-d tensor,[num_anchors,2] 2 for fg and bg
    :return: loss,1-D tensor
    """
    logit_max = tf.reduce_max(predict_logits)
    loss = tf.log(tf.reduce_sum(tf.exp(predict_logits - logit_max), 1, keepdims=True)) + logit_max
    cls_indice = tf.where(cls_target >= 1)
    anchor_cls_logit = tf.gather_nd(predict_logits, cls_indice)
    loss = tf.squeeze(loss)
    print(loss,anchor_cls_logit)
    loss = loss - anchor_cls_logit
    return loss


def hard_neg_mining(cls_target, predict_logits):
    """
    select top 80 negtive anchors the contribute most loss to do the backward pass
    :param cls_target:2-d tensor,[num_anchors,2 ]; 1,0 for pos,neg anchors respectively
    :param predict_logits: 2-d tensor,[num_anchors,2] 2 for fg and bg
    :return:
    """
    loss_to_rank = log_sum_exp(cls_target, predict_logits)
    loss_to_rank = tf.cast(cls_target[:, 0], tf.float64) * loss_to_rank  # set loss for pos anchor to 0
    _, neg_ind = tf.nn.top_k(loss_to_rank, 2)
    pos_ind = tf.squeeze(tf.where(cls_target[:, 1] >= 1))
    pos_ind = tf.cast(pos_ind, tf.int64)
    neg_ind = tf.cast(neg_ind, tf.int64)
    all_ind = tf.concat([pos_ind, neg_ind], axis=0)
    cls_target = tf.gather(cls_target,all_ind)
    predict_logits = tf.gather(predict_logits,all_ind)
    return cls_target, predict_logits


def cls_loss(cls_target, predict_logits):
    """
    :param cls_target:2-d array,[batch_num,num_anchors]; 1,-1,0 for pos,neg and un-train anchors respectively
    :param predict_logits: 3-d array,[batch_num,num_anchors,2] fg or bg
    :return: classification loss of training anchors
    """
    # remove un-trained anchors from cls_target and make cls_target to one-hot format
    train_indices = tf.where(tf.not_equal(cls_target, 0))
    cls_target = tf.gather_nd(cls_target, train_indices)
    cls_target = tf.cast(cls_target, dtype=tf.int32)
    cls_target = tf.one_hot(cls_target, depth=conf.num_class)

    # remove un-trained anchors from pred logit
    logit0 = tf.gather_nd(predict_logits[..., 0], train_indices)
    logit1 = tf.gather_nd(predict_logits[..., 1], train_indices)
    predict_logits = tf.stack([logit0, logit1], axis=1)

    # change negative tag from -1 to 0
    cls_target = tf.where(cls_target > 0, cls_target, tf.zeros_like(cls_target))

    # hard negative anchor mining
    if conf.hard_negative_mining:
        cls_target, predict_logits = hard_neg_mining(cls_target, predict_logits)

    #  calculate the loss
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=cls_target, logits=predict_logits)
    loss = K.mean(loss)
    return loss


def regr_loss(reg_target, predict_deltas, cls_target):
    """
    :param reg_target: 2-d array,[batch_num,num_anchors,(dy,dx,dz,dh)]fg or bg
    :param predict_deltas: 2-d array,[batch_num,num_anchors,(dy,dx,dz,dh)] fg or bg
    :param cls_target: 2-d array,[batch_num,num_anchors]; 1,-1,0 for pos,neg and un-train anchors respectively
    :return:
    """
    # remove un-trained anchors
    pos_indice = tf.where(tf.equal(cls_target, 1))
    reg_target = tf.gather_nd(reg_target, pos_indice)
    pred_deltas = tf.gather_nd(predict_deltas, pos_indice)
    diff = tf.abs(reg_target - pred_deltas)
    smooth_loss = tf.where(diff < 1, tf.pow(diff, 2) * 0.5, diff - 0.5)
    loss = K.mean(smooth_loss)
    return loss


def progressive_anchor_loss(e_reg_targets, e_cls_targets, o_reg_targets, o_cls_targets,
                            fs_cls, fs_regr, ss_cls, ss_regr):
    fs_cls_loss = cls_loss(o_cls_targets, fs_cls)
    ss_cls_loss = cls_loss(e_cls_targets, ss_cls)
    fs_regr_loss = regr_loss(o_reg_targets, fs_regr, o_cls_targets)
    ss_regr_loss = regr_loss(e_reg_targets, ss_regr, e_cls_targets)

    # fs_cls_loss = K.print_tensor(fs_cls_loss, message='fs_cls_loss = ')
    # ss_cls_loss = K.print_tensor(ss_cls_loss, message='ss_cls_loss = ')
    # fs_regr_loss = K.print_tensor(fs_regr_loss, message='fs_regr_loss = ')
    # ss_regr_loss = K.print_tensor(ss_regr_loss, message='ss_regr_loss = ')

    total_loss = fs_cls_loss + ss_cls_loss + fs_regr_loss + ss_regr_loss
    # total_loss = K.print_tensor(total_loss, message='total_loss = ')
    return total_loss


if __name__=='__main__':
    import numpy as np
    cls_target = np.random.randint(-1, 2, [2, 8])
    predict_logits = np.random.randint(-1, 10, [2, 8, 2])
    cls_target = tf.constant(cls_target)
    predict_logits = tf.constant(predict_logits, tf.float64)
    see = cls_loss(cls_target, predict_logits)
    to_run = [see]
    sess = tf.Session()
    for i in to_run:
        print(i, '\n', sess.run(i))


Tensor("Squeeze:0", dtype=float64) Tensor("GatherNd_3:0", shape=(?,), dtype=float64)


InvalidArgumentError: Incompatible shapes: [11] vs. [7]
	 [[Node: sub_1 = Sub[T=DT_DOUBLE, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Squeeze, GatherNd_3)]]

Caused by op 'sub_1', defined at:
  File "/root/anaconda3/envs/keras/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/root/anaconda3/envs/keras/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/root/anaconda3/envs/keras/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/root/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/root/anaconda3/envs/keras/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File

In [5]:
# -*- coding:utf-8 _*-
"""
@author: danna.li
@date: 2019/4/25
@file: loss_func.py
@description:
"""

import tensorflow as tf
import keras.backend as K
from dual_conf import current_config as conf


def log_sum_exp(cls_target, predict_logits):
    """un-averaged confidence loss across all examples in a batch.
    :param cls_target: 2-d tensor,[num_anchors,2 ]; 1,0 for pos,neg anchors respectively
    :param predict_logits: 2-d tensor,[num_anchors,2] 2 for fg and bg
    :return: loss,1-D tensor
    """
    logit_max = tf.reduce_max(predict_logits)
    loss = tf.log(tf.reduce_sum(tf.exp(predict_logits - logit_max), 1, keepdims=True)) + logit_max
    cls_indice = tf.where(cls_target >= 1)
    anchor_cls_logit = tf.gather_nd(predict_logits, cls_indice)
    loss = tf.squeeze(loss)
    loss = loss - anchor_cls_logit
    return loss


def hard_neg_mining(cls_target, predict_logits):
    """
    select top 80 negtive anchors the contribute most loss to do the backward pass
    :param cls_target:2-d tensor,[num_anchors,2 ]; 1,0 for pos,neg anchors respectively
    :param predict_logits: 2-d tensor,[num_anchors,2] 2 for fg and bg
    :return:
    """
    loss_to_rank = log_sum_exp(cls_target, predict_logits)
    print(loss_to_rank,cls_target[:, 0])
    loss_to_rank = cls_target[:, 0] * loss_to_rank  # set loss for pos anchor to 0
    _, neg_ind = tf.nn.top_k(loss_to_rank, 80)
    pos_ind = tf.squeeze(tf.where(cls_target[:, 1] >= 1))
    pos_ind = tf.cast(pos_ind, tf.int64)
    neg_ind = tf.cast(neg_ind, tf.int64)
    all_ind = tf.concat([pos_ind, neg_ind], axis=0)
    cls_target = tf.gather(cls_target, all_ind)
    predict_logits = tf.gather(predict_logits, all_ind)
    return cls_target, predict_logits


def cls_loss(cls_target, predict_logits):
    """
    :param cls_target:2-d array,[batch_num,num_anchors]; 1,-1,0 for pos,neg and un-train anchors respectively
    :param predict_logits: 3-d array,[batch_num,num_anchors,2] fg or bg
    :return: classification loss of training anchors
    """
    # remove un-trained anchors from cls_target and make cls_target to one-hot format
    train_indices = tf.where(tf.not_equal(cls_target, 0))
    cls_target = tf.gather_nd(cls_target, train_indices)
    # change negative tag from -1 to 0
    cls_target = tf.where(cls_target > 0, cls_target, tf.zeros_like(cls_target))
    cls_target = tf.cast(cls_target,tf.int64)
    cls_target = tf.one_hot(cls_target, depth=conf.num_class)

    # remove un-trained anchors from pred logit
    logit0 = tf.gather_nd(predict_logits[..., 0], train_indices)
    logit1 = tf.gather_nd(predict_logits[..., 1], train_indices)
    predict_logits = tf.stack([logit0, logit1], axis=1)
    #predict_logits = tf.cast(predict_logits, dtype=tf.float32)

    # hard negative anchor mining
    if conf.hard_negative_mining:
        cls_target, predict_logits = hard_neg_mining(cls_target, predict_logits)

    #  calculate the loss
    print(cls_target,predict_logits)
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=cls_target, logits=predict_logits)
    loss = K.mean(loss)
    return loss







if __name__ == '__main__':
    import numpy as np
    cls_target = np.random.randint(-1, 2, [2, 8])
    predict_logits = np.random.randint(-1, 10, [2, 8, 2])
    cls_target = tf.constant(cls_target)
    predict_logits = tf.constant(predict_logits, tf.float64)
    loss = cls_loss(cls_target, predict_logits)
    to_run = [loss]
    sess = tf.Session()
    for i in to_run:
        print(i, '\n', sess.run(i))


Tensor("sub_5:0", dtype=float64) Tensor("strided_slice_9:0", shape=(?,), dtype=float32)


TypeError: Input 'y' of 'Mul' Op has type float64 that does not match type float32 of argument 'x'.

In [ ]:
import torch

In [ ]:
def log_sum_exp(x):
    """Utility function for computing log_sum_exp while determining
    This will be used to determine unaveraged confidence loss across
    all examples in a batch.
    Args:
        x (Variable(tensor)): conf_preds from conf layers
    """
    x_max = x.data.max()
    return torch.log(torch.sum(torch.exp(x - x_max), 1, keepdim=True)) + x_max



In [ ]:
x = torch.Tensor([[1,2],[1000,4],[3,2]])

In [ ]:
log_sum_exp(x)

In [ ]:
conf_t = torch.Tensor([0,0,1]).long()

In [ ]:
conf_t.view(-1, 1)

In [ ]:
x.gather(1, conf_t.view(-1, 1))

In [ ]:
log_sum_exp(x)-x.gather(1, conf_t.view(-1, 1))

In [ ]:
import sys
sys.path.append('/opt/code/face/dual_shot')
import numpy as np
from net.dual_shot import test_net
from prepare_data.generator import image_reader,augment
from prepare_data.augment import Augmentation
from prepare_data.widerface import WIDERFaceDetection
from dual_conf import current_config as conf
import tensorflow as tf

import tensorflow as tf 
config = tf.ConfigProto()
config.gpu_options.allow_growth= True
sess = tf.Session(config=config)
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
test_set = WIDERFaceDetection(conf.data_path, 'train', None, None)
image, gts, labels = image_reader(test_set)
sub_img, gt_in_crop, sparse_label = augment(image, gts, labels, test_set)

In [ ]:
image = tf.Variable(np.expand_dims(sub_img,0))
ss_cls, _ = test_net(image)

In [ ]:
sess.run(tf.global_variables_initializer())
ss_cls = ss_cls.eval(session=sess)
ss_cls = np.squeeze(ss_cls)

In [ ]:
def log_sum_exp(pred_logit,anchor_cls):
    """ determine un-averaged confidence loss across all examples in a batch.
    Args:
        x (Variable(tensor)): conf_preds from conf layers
    """
    logit_max = np.max(pred_logit)
    loss = np.log(np.sum(np.exp(pred_logit - logit_max), 1)+ logit_max) 
    anchor_cls_logit = pred_logit[range(len(anchor_cls)),anchor_cls]
    loss = loss - anchor_cls_logit
    return loss

In [ ]:
loss = log_sum_exp(ss_cls,anchor_cls)

In [ ]:
pos_ind = np.where(np.equal(anchor_cls, 1))[0]
loss[pos_ind]=0

In [ ]:
loss

In [ ]:
(-loss).sort()
loss

In [ ]:
_, loss_idx = loss.sort()
_, idx_rank = loss_idx.sort(1)

In [ ]:

pos_ind

In [ ]:
ss_cls

In [ ]:
anchor_cls = np.random.randint(0,2,(len(ss_cls),))
anchor_cls

In [ ]:
anchor_cls_logit = ss_cls[range(len(anchor_cls)),anchor_cls]

In [ ]:
np.sum(np.exp(x - x_max), 1)

In [ ]:
def hard_neg_mining(cls_target, predict_logits):
    """
    select top 80 negtive anchors the contribute most loss to do the backward pass
    :param cls_target:2-d tensor,[num_anchors,2 ]; 1,0 for pos,neg anchors respectively
    :param predict_logits: 2-d tensor,[num_anchors,2] 2 for fg and bg
    :return:
    """
    loss_to_rank = log_sum_exp(cls_target, predict_logits)
    loss_to_rank = cls_target[:, 0] * loss_to_rank  # set loss for pos anchor to 0
    _, neg_ind = tf.nn.top_k(loss_to_rank, 80)
    pos_ind = tf.squeeze(tf.where(cls_target[:, 1] >= 1))
    pos_ind = tf.cast(pos_ind, tf.int64)
    neg_ind = tf.cast(neg_ind, tf.int64)
    all_ind = tf.concat([pos_ind, neg_ind], axis=0)
    cls_target = tf.gather(cls_target, all_ind)
    predict_logits = tf.gather(predict_logits, all_ind)
    return cls_target, predict_logits